In [4]:
!pip install langchain langchain-huggingface langchain_community faiss-cpu

In [9]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
model_name = "HooshvareLab/bert-base-parsbert-uncased"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf_embedding = HuggingFaceEmbeddings(
model_name=model_name,
model_kwargs=model_kwargs,
encode_kwargs=encode_kwargs
)

In [10]:
text = "همه ی افراد می توانند از امکانات هوش مصنوعی استفاده کنند"
embed = hf_embedding.embed_query(text)
print(len(embed))


768


In [10]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
# چون نتونستم مدل HooshvareLab/bert-base-parsbert-uncased رو نصب کنم به جاش از یه مدل دیگه استفاده می کنم
# نصب مدل heydariAI/persian-embeddings برای embeddings
model_name = "heydariAI/persian-embeddings"
hf_embeddings = HuggingFaceEmbeddings(model_name=model_name)

# متن نمونه
text = """
هوش مصنوعی یکی از بزرگ‌ترین دستاوردهای فناوری در دهه‌های اخیر است که تأثیرات قابل‌توجهی در جنبه‌های مختلف زندگی بشر داشته است. این فناوری با استفاده از روش‌هایی مانند یادگیری عمیق و شبکه‌های عصبی مصنوعی قادر است تا وظایف پیچیده‌ای مانند تحلیل داده، تشخیص تصویر، پردازش زبان طبیعی و حتی تصمیم‌گیری را انجام دهد. در صنعت پزشکی، هوش مصنوعی نقشی اساسی در تشخیص بیماری‌ها، پیش‌بینی روند درمان و حتی توسعه داروهای جدید ایفا می‌کند. سیستم‌های مبتنی بر یادگیری ماشینی قادرند با تحلیل داده‌های بیماران، تشخیص‌های سریع‌تر و دقیق‌تری ارائه دهند که به افزایش کیفیت خدمات درمانی منجر می‌شود.

در صنعت خودروسازی، هوش مصنوعی تکنولوژی خودروهای خودران را ممکن ساخته است. این خودروها با استفاده از سنسورها، دوربین‌ها و الگوریتم‌های پیشرفته می‌توانند بدون نیاز به راننده در جاده حرکت کنند. علاوه بر این، استفاده از هوش مصنوعی در خطوط تولید کارخانه‌ها باعث افزایش بهره‌وری و کاهش خطاهای انسانی شده است.

در زمینه آموزش، هوش مصنوعی نقش پررنگی در سفارشی‌سازی فرآیند یادگیری برای دانش‌آموزان دارد. سیستم‌های آموزشی مبتنی بر هوش مصنوعی می‌توانند محتوا و روش تدریس را بر اساس نیازها و توانایی‌های هر دانش‌آموز تنظیم کنند. این موضوع باعث افزایش اثربخشی آموزش و انگیزه بیشتر در یادگیری می‌شود.

علاوه بر این‌ها، کاربردهای هوش مصنوعی در حوزه‌هایی مانند کشاورزی، مالی، امنیت سایبری و حتی سرگرمی نیز رشد چشمگیری داشته است. در کشاورزی، فناوری‌های هوشمند می‌توانند به تحلیل شرایط خاک و پیش‌بینی آب و هوا کمک کرده و بهره‌وری کشاورزان را افزایش دهند. در دنیای مالی، الگوریتم‌های مبتنی بر یادگیری ماشین به تحلیل بازارهای مالی، پیش‌بینی نوسانات بازار و شناسایی رفتارهای مشکوک در تراکنش‌های مالی می‌پردازند.

هوش مصنوعی همچنین وارد دنیای هنر و موسیقی شده است. از طراحی گرافیکی و تولید آثار هنری دیجیتال گرفته تا ساخت موسیقی با استفاده از الگوریتم‌های پیشرفته، این فناوری انقلابی در خلاقیت هنری به وجود آورده است. نرم‌افزارهای مبتنی بر هوش مصنوعی می‌توانند موسیقی‌های سفارشی براساس سبک و سلیقه کاربران تولید کنند و حتی در تحلیل قطعات موسیقی کلاسیک به موزیسین‌ها کمک کنند. در زمینه هنرهای بصری، استفاده از شبکه‌های مولد عمیق (GANs) به هنرمندان امکان می‌دهد تا تصاویر و نقاشی‌های بی‌نظیری خلق کنند.

در حوزه مهندسی، هوش مصنوعی توانسته است فرایندهای طراحی، شبیه‌سازی و تولید را متحول کند. از طراحی سازه‌های پیچیده گرفته تا تحلیل داده‌های مربوط به عملکرد ماشین‌آلات صنعتی، این فناوری نقش مؤثری در کاهش هزینه‌ها و افزایش دقت دارد. همچنین، در مهندسی نرم‌افزار، استفاده از مدل‌های یادگیری ماشینی در تشخیص و رفع خطاهای کد، توسعه برنامه‌ها را سریع‌تر و کارآمدتر کرده است.

در مهندسی پزشکی، هوش مصنوعی در طراحی دستگاه‌های پیشرفته پزشکی و همچنین تحلیل داده‌های کلینیکی نقش اساسی ایفا می‌کند. این فناوری به توسعه تکنولوژی‌هایی مانند پروتزهای هوشمند، دستگاه‌های تصویربرداری پزشکی (مانند MRI و CT Scan)، و سیستم‌های تشخیص بیماری کمک کرده است. الگوریتم‌های پیشرفته هوش مصنوعی می‌توانند داده‌های مربوط به بدن انسان را با دقت بالا تحلیل کرده و به پزشکان در تصمیم‌گیری بهتر کمک کنند. علاوه بر این، ربات‌های جراحی مبتنی بر هوش مصنوعی اکنون در بسیاری از بیمارستان‌ها به‌کار گرفته می‌شوند و جراحی‌هایی دقیق‌تر و کم‌خطرتر را ممکن می‌سازند.

با تمام این مزایا، هوش مصنوعی چالش‌ها و نگرانی‌هایی را نیز به همراه داشته است. مسائل مربوط به حریم خصوصی، امنیت داده‌ها و اخلاقیات استفاده از این فناوری از جمله موضوعاتی هستند که باید به دقت مورد بررسی قرار گیرند. علاوه بر این، نیاز به بهبود شفافیت الگوریتم‌های هوش مصنوعی و جلوگیری از بروز سوگیری‌ها در تصمیمات این سیستم‌ها از دیگر چالش‌های مهم در این حوزه است.

"""

# تقسیم متن به chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # حداکثر تعداد کاراکتر در هر chunk
    chunk_overlap=50,  # میزان همپوشانی بین chunk‌ها
)
chunks = text_splitter.split_text(text)

# تبدیل chunks به اسناد (Documents)
documents = [Document(page_content=chunk) for chunk in chunks]

# ایجاد یک FAISS Vectorstore
vectorstore = FAISS.from_documents(documents, hf_embeddings)

# ذخیره FAISS index در فایل (اختیاری)
faiss_index_path = "faiss_index"
vectorstore.save_local(faiss_index_path)

print("FAISS Vectorstore ساخته شد و ذخیره شد.")

# جستجوی معنایی
query = "کاربردهای هوش مصنوعی در پزشکی چیست؟"
results = vectorstore.similarity_search(query, k=3)  #بهترین k نتیجه را نشان می دهد برای متن بالا وقتی  k=2  را انتخاب کردم دوتا از مرتبط ترین نتایج را نشان داد ولی وقتی  k=5 قرار دادم سه نتیجه اول را مرتبط و مابقی را غیر مرتبط نشان داد که
#که طبیعی به نظر میرسد چون بر اساس بردار کار می کند و نتایج را ولو غیر مرتبط باشند نشان می دهد مگر اینکه از روش های کنترلی استفاده کنیم

for result in results:
    print("Matched Text:", result.page_content)


FAISS Vectorstore ساخته شد و ذخیره شد.
Matched Text: در مهندسی پزشکی، هوش مصنوعی در طراحی دستگاه‌های پیشرفته پزشکی و همچنین تحلیل داده‌های کلینیکی نقش اساسی ایفا می‌کند. این فناوری به توسعه تکنولوژی‌هایی مانند پروتزهای هوشمند، دستگاه‌های تصویربرداری پزشکی (مانند MRI و CT Scan)، و سیستم‌های تشخیص بیماری کمک کرده است. الگوریتم‌های پیشرفته هوش مصنوعی می‌توانند داده‌های مربوط به بدن انسان را با دقت بالا تحلیل کرده و به پزشکان در تصمیم‌گیری بهتر کمک کنند. علاوه بر این، ربات‌های جراحی مبتنی بر هوش مصنوعی اکنون در بسیاری از بیمارستان‌ها به‌کار گرفته
Matched Text: هوش مصنوعی یکی از بزرگ‌ترین دستاوردهای فناوری در دهه‌های اخیر است که تأثیرات قابل‌توجهی در جنبه‌های مختلف زندگی بشر داشته است. این فناوری با استفاده از روش‌هایی مانند یادگیری عمیق و شبکه‌های عصبی مصنوعی قادر است تا وظایف پیچیده‌ای مانند تحلیل داده، تشخیص تصویر، پردازش زبان طبیعی و حتی تصمیم‌گیری را انجام دهد. در صنعت پزشکی، هوش مصنوعی نقشی اساسی در تشخیص بیماری‌ها، پیش‌بینی روند درمان و حتی توسعه داروهای جدید ایفا می‌کند. سیستم‌های م

In [11]:
import requests
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# داده نمونه
text = """
هوش مصنوعی یکی از بزرگ‌ترین دستاوردهای فناوری در دهه‌های اخیر است که تأثیرات قابل‌توجهی در جنبه‌های مختلف زندگی بشر داشته است. این فناوری با استفاده از روش‌هایی مانند یادگیری عمیق و شبکه‌های عصبی مصنوعی قادر است تا وظایف پیچیده‌ای مانند تحلیل داده، تشخیص تصویر، پردازش زبان طبیعی و حتی تصمیم‌گیری را انجام دهد. در صنعت پزشکی، هوش مصنوعی نقشی اساسی در تشخیص بیماری‌ها، پیش‌بینی روند درمان و حتی توسعه داروهای جدید ایفا می‌کند. سیستم‌های مبتنی بر یادگیری ماشینی قادرند با تحلیل داده‌های بیماران، تشخیص‌های سریع‌تر و دقیق‌تری ارائه دهند که به افزایش کیفیت خدمات درمانی منجر می‌شود.

در صنعت خودروسازی، هوش مصنوعی تکنولوژی خودروهای خودران را ممکن ساخته است. این خودروها با استفاده از سنسورها، دوربین‌ها و الگوریتم‌های پیشرفته می‌توانند بدون نیاز به راننده در جاده حرکت کنند. علاوه بر این، استفاده از هوش مصنوعی در خطوط تولید کارخانه‌ها باعث افزایش بهره‌وری و کاهش خطاهای انسانی شده است.

در زمینه آموزش، هوش مصنوعی نقش پررنگی در سفارشی‌سازی فرآیند یادگیری برای دانش‌آموزان دارد. سیستم‌های آموزشی مبتنی بر هوش مصنوعی می‌توانند محتوا و روش تدریس را بر اساس نیازها و توانایی‌های هر دانش‌آموز تنظیم کنند. این موضوع باعث افزایش اثربخشی آموزش و انگیزه بیشتر در یادگیری می‌شود.

علاوه بر این‌ها، کاربردهای هوش مصنوعی در حوزه‌هایی مانند کشاورزی، مالی، امنیت سایبری و حتی سرگرمی نیز رشد چشمگیری داشته است. در کشاورزی، فناوری‌های هوشمند می‌توانند به تحلیل شرایط خاک و پیش‌بینی آب و هوا کمک کرده و بهره‌وری کشاورزان را افزایش دهند. در دنیای مالی، الگوریتم‌های مبتنی بر یادگیری ماشین به تحلیل بازارهای مالی، پیش‌بینی نوسانات بازار و شناسایی رفتارهای مشکوک در تراکنش‌های مالی می‌پردازند.

هوش مصنوعی همچنین وارد دنیای هنر و موسیقی شده است. از طراحی گرافیکی و تولید آثار هنری دیجیتال گرفته تا ساخت موسیقی با استفاده از الگوریتم‌های پیشرفته، این فناوری انقلابی در خلاقیت هنری به وجود آورده است. نرم‌افزارهای مبتنی بر هوش مصنوعی می‌توانند موسیقی‌های سفارشی براساس سبک و سلیقه کاربران تولید کنند و حتی در تحلیل قطعات موسیقی کلاسیک به موزیسین‌ها کمک کنند. در زمینه هنرهای بصری، استفاده از شبکه‌های مولد عمیق (GANs) به هنرمندان امکان می‌دهد تا تصاویر و نقاشی‌های بی‌نظیری خلق کنند.

در حوزه مهندسی، هوش مصنوعی توانسته است فرایندهای طراحی، شبیه‌سازی و تولید را متحول کند. از طراحی سازه‌های پیچیده گرفته تا تحلیل داده‌های مربوط به عملکرد ماشین‌آلات صنعتی، این فناوری نقش مؤثری در کاهش هزینه‌ها و افزایش دقت دارد. همچنین، در مهندسی نرم‌افزار، استفاده از مدل‌های یادگیری ماشینی در تشخیص و رفع خطاهای کد، توسعه برنامه‌ها را سریع‌تر و کارآمدتر کرده است.

در مهندسی پزشکی، هوش مصنوعی در طراحی دستگاه‌های پیشرفته پزشکی و همچنین تحلیل داده‌های کلینیکی نقش اساسی ایفا می‌کند. این فناوری به توسعه تکنولوژی‌هایی مانند پروتزهای هوشمند، دستگاه‌های تصویربرداری پزشکی (مانند MRI و CT Scan)، و سیستم‌های تشخیص بیماری کمک کرده است. الگوریتم‌های پیشرفته هوش مصنوعی می‌توانند داده‌های مربوط به بدن انسان را با دقت بالا تحلیل کرده و به پزشکان در تصمیم‌گیری بهتر کمک کنند. علاوه بر این، ربات‌های جراحی مبتنی بر هوش مصنوعی اکنون در بسیاری از بیمارستان‌ها به‌کار گرفته می‌شوند و جراحی‌هایی دقیق‌تر و کم‌خطرتر را ممکن می‌سازند.

با تمام این مزایا، هوش مصنوعی چالش‌ها و نگرانی‌هایی را نیز به همراه داشته است. مسائل مربوط به حریم خصوصی، امنیت داده‌ها و اخلاقیات استفاده از این فناوری از جمله موضوعاتی هستند که باید به دقت مورد بررسی قرار گیرند. علاوه بر این، نیاز به بهبود شفافیت الگوریتم‌های هوش مصنوعی و جلوگیری از بروز سوگیری‌ها در تصمیمات این سیستم‌ها از دیگر چالش‌های مهم در این حوزه است.

"""

query = "کاربردهای هوش مصنوعی در پزشکی چیست؟"

# تقسیم متن به chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_text(text)
documents = [Document(page_content=chunk) for chunk in chunks]

# کلاس سفارشی برای ارسال درخواست به API سفارشی
class CustomOpenAIEmbeddings:
    def __init__(self, base_url, api_key):
        self.base_url = base_url
        self.api_key = api_key

    def embed_documents(self, texts):
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        embeddings = []
        for text in texts:
            response = requests.post(
                f"{self.base_url}/embeddings",
                headers=headers,
                json={"model": "text-embedding-ada-002", "input": text},
            )
            if response.status_code == 200:
                data = response.json()
                embeddings.append(data["data"][0]["embedding"])
            else:
                raise ValueError(f"Error {response.status_code}: {response.text}")
        return embeddings

# تنظیم Embeddings با URL و کلید API دلخواه
custom_openai_embedding = CustomOpenAIEmbeddings(
    base_url="https://api.avalai.ir/v1",
    api_key="aa-Z5qRX4U5RnK8q5Aig4RVdOOpIZ9ZUY0lX44Bc2MBnNGfGbHM"
)

# ایجاد بردارهای Embedding برای اسناد
embeddings = custom_openai_embedding.embed_documents([doc.page_content for doc in documents])

# ایجاد FAISS VectorStore
import faiss
dimension = len(embeddings[0])  # تعداد ابعاد embedding
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings, dtype="float32"))

# جستجوی معنایی
query_embedding = custom_openai_embedding.embed_documents([query])[0]
D, I = index.search(np.array([query_embedding], dtype="float32"), k=3)  # جستجوی 3 نتیجه مرتبط

# نمایش نتایج
print("نتایج جستجو:")
for i in I[0]:
    print(f"* {documents[i].page_content}")


نتایج جستجو:
* در مهندسی پزشکی، هوش مصنوعی در طراحی دستگاه‌های پیشرفته پزشکی و همچنین تحلیل داده‌های کلینیکی نقش اساسی ایفا می‌کند. این فناوری به توسعه تکنولوژی‌هایی مانند پروتزهای هوشمند، دستگاه‌های تصویربرداری پزشکی (مانند MRI و CT Scan)، و سیستم‌های تشخیص بیماری کمک کرده است. الگوریتم‌های پیشرفته هوش مصنوعی می‌توانند داده‌های مربوط به بدن انسان را با دقت بالا تحلیل کرده و به پزشکان در تصمیم‌گیری بهتر کمک کنند. علاوه بر این، ربات‌های جراحی مبتنی بر هوش مصنوعی اکنون در بسیاری از بیمارستان‌ها به‌کار گرفته
* هوش مصنوعی یکی از بزرگ‌ترین دستاوردهای فناوری در دهه‌های اخیر است که تأثیرات قابل‌توجهی در جنبه‌های مختلف زندگی بشر داشته است. این فناوری با استفاده از روش‌هایی مانند یادگیری عمیق و شبکه‌های عصبی مصنوعی قادر است تا وظایف پیچیده‌ای مانند تحلیل داده، تشخیص تصویر، پردازش زبان طبیعی و حتی تصمیم‌گیری را انجام دهد. در صنعت پزشکی، هوش مصنوعی نقشی اساسی در تشخیص بیماری‌ها، پیش‌بینی روند درمان و حتی توسعه داروهای جدید ایفا می‌کند. سیستم‌های مبتنی بر یادگیری ماشینی قادرند با تحلیل داده‌های بی

In [ ]:
#به نظر میرسه که مدل لوکال امبدینگ کمی بهتر عمل کرده و لااقل در مورد نتیجه ی سوم میبینیم متنی که امبدینگ به ما داده مرتبط تر از چیزی هست که  اپن ای آی  برگردونده و این نتیجه با وجود یکسان بودن متن و پارامترهای k , chunk_size , chunk_overlap  به دست اومده

به نظر میرسه که مدل لوکال امبدینگ کمی بهتر عمل کرده و لااقل در مورد نتیجه ی سوم میبینیم متنی که امبدینگ به ما داده مرتبط تر از چیزی هست که open ai  برگردونده و این نتیجه با وجود یکسان بودن متن و پارامترهای k , chunk_size , chunk_overlap  به دست اومده